In [ ]:
import pandas as pd
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pickle

from random import randint
from numpy import array
from numpy import argmax
from random import randint
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

from math import sin
from math import pi
from math import exp
from random import randint
from random import uniform

from numpy import array
from math import ceil
from math import log10
from keras.layers import TimeDistributed
from keras.layers import RepeatVector

from random import random
from numpy import cumsum
from numpy import array_equal
from keras.layers import Bidirectional

from sklearn.preprocessing import MinMaxScaler

import re

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 500)

In [ ]:
path = './Datasets/'
model_path = './Models/'

In [ ]:
# Read in config file - column list

cumsum_cols_p = open(path + 'cumsum_cols.pkl', 'rb')
cumsum_cols = pickle.load(cumsum_cols_p)

In [ ]:
# Eval transformation functions

def most_common(lst): 
    return max(set(lst), key = lst.count)

def reveal_bias(url):
    path = url.split('/', 1)[1]
    apps = re.sub('[/-]', ' ',path).split()[:-1]
    return most_common(apps)

def add_bias_labels(col_lst):
    for i in range(len(col_lst)):
        col_lst[i] = col_lst[i] + "-bias-" + reveal_bias(col_lst[i])
    return col_lst

def create_reqset_dict(col_list, req_list):
    reqset_dict = dict(zip(add_bias_labels(col_list),req_list))
    return reqset_dict

def get_reqset_bias(dictionary):
    bias_2 = sum([v for k,v in dictionary.items() if "-bias-2" in k])
    bias_3 = sum([v for k,v in dictionary.items() if "-bias-3" in k])
    bias_4 = sum([v for k,v in dictionary.items() if "-bias-4" in k])
    
    return (bias_2, bias_3, bias_4)

def get_app_bias_error(X_test, y_test, model):
    error_vectors_app_bias = list()
    for i in range(len(X_test)):
        predict_dict = create_reqset_dict(cumsum_cols, model.predict(X_test[i].reshape(1,1000,129))[0])
        predict_bias = get_reqset_bias(predict_dict)

        actual_dict = create_reqset_dict(cumsum_cols, y_test[i])
        actual_bias = get_reqset_bias(actual_dict)

        error_vectors_app_bias.append(np.absolute(np.array(actual_bias) - np.array(predict_bias)))
        print("step " + str(i) + "done")
        print(str(len(X_test) - i) + " steps remaining")
    return error_vectors_app_bias

In [ ]:
# Read in prepared datasets

X_val_p = open(path + 'X_val.pkl', 'rb')
X_val = pickle.load(X_val_p)

sub_y_val_p = open(path + 'y_val.pkl', 'rb')
sub_y_val = pickle.load(sub_y_val_p)

X_train_p = open(path + 'X_train.pkl', 'rb')
X = pickle.load(X_train_p)

y_train_p = open(path + 'y_train.pkl', 'rb')
sub_y = pickle.load(y_train_p)

X_test_p = open(path + 'X_test.pkl', 'rb')
X_test = pickle.load(X_test_p)

y_test_p = open(path + 'y_test.pkl', 'rb')
sub_y_test = pickle.load(y_test_p)

In [ ]:
# define model - based on chosen model from grid search

model = Sequential()
model.add(LSTM(125, return_sequences=True, input_shape=(1000, 129)))
model.add(LSTM(125, return_sequences=True))
model.add(LSTM(125))
model.add(Dense(129))
model.compile(loss='mae', optimizer=keras.optimizers.Adam(lr=0.01))

# fit model

history = model.fit(X, sub_y, batch_size=128, epochs=1, validation_data=(X_val, sub_y_val), shuffle=False)

# fit model
history = model.fit(X, sub_y, batch_size=32, epochs=1)

In [ ]:
# evaluate model by MAE

loss = model.evaluate(X_test, sub_y_test, verbose=0)
loss_string = 'MAE: %f' % loss

# compared predicted request set "app bias" with actual "app bias"

app_bias_list = get_app_bias_error(X_test, sub_y_test, model)

app_bias_df = pd.DataFrame(app_bias_list)

app_bias_mean = app_bias_df.mean().mean()

# save model data in dictionary

model_dict = {
    "model-json": model.to_json(),
    "model-history": history.history,
    "MAE-holdout-set": loss_string,
    "app-bias-mean-holdout-set": app_bias_mean,
    "model-object": model
}

print(model_dict)

with open(model_path + 'LSTM_model.pkl', 'wb') as f:
    pickle.dump(model_dict, f)